In [1]:
import paddle
from paddle.nn import Linear
import paddle.nn.functional as F
import numpy as np
import os
import random

In [14]:
datafile = './work/housing.data'
data = np.fromfile(datafile, sep=' ', dtype=np.float32)

In [15]:
data.shape
np.random.shuffle(data)

In [2]:
def load_data():
    # 从文件导入数据
    datafile = './work/housing.data'
    data = np.fromfile(datafile, sep=' ', dtype=np.float32)

    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \
                      'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)

    # 将原始数据进行Reshape，变成[N, 14]这样的形状
    data = data.reshape([data.shape[0] // feature_num, feature_num])

    # 将原数据集拆分成训练集和测试集
    # 这里使用80%的数据做训练，20%的数据做测试
    # 测试集和训练集必须是没有交集的
    np.random.shuffle(data) # 打乱data的顺序
    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]


    # 计算train数据集的最大值，最小值，平均值
    maximums, minimums, avgs = training_data.max(axis=0), training_data.min(axis=0), \
                                 training_data.sum(axis=0) / training_data.shape[0]
    
    # 记录数据的归一化参数，在预测时对数据做归一化
    global max_values
    global min_values
    global avg_values
    max_values = maximums
    min_values = minimums
    avg_values = avgs

    # 对数据进行归一化处理
    for i in range(feature_num):
        data[:, i] = (data[:, i] - avgs[i]) / (maximums[i] - minimums[i])

    # 训练集和测试集的划分比例
    training_data = data[:offset]
    test_data = data[offset:]
    return training_data, test_data

In [3]:
# 定义网络结构-模型设计
class Regressor(paddle.nn.Layer):
    def __init__(self):
        super(Regressor, self).__init__()

        self.fc = Linear(in_features=13,out_features=1)

    def forward(self,inputs):
        x = self.fc(inputs)
        return x

In [8]:
# 训练配置:设定优化器（参数更新策略）及损失函数
model = Regressor() #创建模型
model.train() # 开启模型的训练模式
# 定义优化算法, 这里使用梯度下降
# model.parameters(), 返回模型的参数
opt = paddle.optimizer.SGD(learning_rate=0.01,parameters=model.parameters())
# 加载数据
train_data,test_data = load_data()

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
# 训练过程
EPOCH_NUM = 10
batch_size = 10

for epoch_id in range(EPOCH_NUM):
    mini_batchs = [train_data[k,k+batch_size] for k in range(0,len(train_data),batch_size)]

    for iter_id,mini_batch in enumerate(mini_batchs):
        x =mini_batch[:,:-1]
        y = mini_batch[:,-1:]

        x = paddle.to_tensor(x)
        y = paddle.to_tensor(y)

        predicts = model(x) # 前向计算，model的训练模型已经开启，直接传值相当于调用forward函数

        loss = F.square_error_cost(predicts,label=y) #计算每一个的损失
        avg_cost = paddle.mean(loss) # 计算平均损失

        avg_cost.backward() # 反向传播，相当于求解微分增量的过程
        #根据优化函数，执行参数更新
        opt.step()
        # 清除梯度，即：清除求解的微分增量信息
        opt.clear_grad()

    